In [ ]:
!pip install tokengrams

In [22]:
# Create a dataset directly from the .txt file without loading it into memory
from datasets import load_dataset

dummy_dataset = load_dataset('text', data_files='../data/cleaned_train.txt')

Generating train split: 20550004 examples [00:09, 2104559.69 examples/s]


In [23]:
# Tokenize HF dataset
from tokengrams import tokenize_hf_dataset


tokenize_hf_dataset(
    dataset=dummy_dataset,
    tokenizer=AutoTokenizer.from_pretrained("EleutherAI/pythia-160m"),
    output_path="tinystories.bin",
    text_key="text",
    append_eod=True,
    workers=1,
)

/Users/bjarnihaukurbjarnason/Documents/School/ProjectCourseInDataScience/ngram-llm-analysis/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Tokenizing: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tokenizing: 9020000it [08:58, 16496.36it/s]

KeyboardInterrupt: 

Tokenizing: 9020000it [09:13, 16496.36it/s]

In [11]:
from tokengrams import MemmapIndex

# Create a new index from an on-disk corpus of u16 tokens and save it to a .idx file. 
# Set verbose to true to include a progress bar for the index sort.
index = MemmapIndex.build(
    "tinystories.bin",
    "tinystories.idx",
    vocab=2**16,
    verbose=True
)

# True for any valid index.
print(index.is_sorted())
  
# Get the count of "hello world" in the corpus.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")
print(index.count(tokenizer.encode("hello world")))

# You can now load the index from disk later using __init__
index = MemmapIndex(
    "tinystories.bin",
    "tinystories.idx",
    vocab=2**16
)

Writing indices to disk...
True
Time elapsed: 475.875µs
Sorting indices...
Time elapsed: 9.319792ms
0


In [16]:
# # Count how often each token in the corpus succeeds "hello world".
print(index.count_next(tokenizer.encode("hello world")))
print(index.batch_count_next(
    [tokenizer.encode("hello world"), tokenizer.encode("hello universe")]
))

# # Get smoothed probabilities for query continuations
# print(index.get_smoothed_probs(tokenizer.encode("hello world")))
# print(index.batch_get_smoothed_probs(
#     [tokenizer.encode("hello world"), tokenizer.encode("hello universe")]
# ))

# # Autoregressively sample 10 tokens using 5-gram language statistics. Initial
# # gram statistics are derived from the query, with lower order gram statistics used 
# # until the sequence contains at least 5 tokens.
# print(index.sample_unsmoothed(tokenizer.encode("hello world"), n=5, k=10, num_samples=20))
# print(index.sample_smoothed(tokenizer.encode("hello world"), n=5, k=10, num_samples=20))

# # Query whether the corpus contains "hello world"
# print(index.contains(tokenizer.encode("hello world")))

# # Get all n-grams beginning with "hello world" in the corpus
# print(index.positions(tokenizer.encode("hello world")))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
index.get_smoothed_probs(tokenizer.encode("hello world"))

[3.3104977640676076e-07,
 8.168084317448003e-18,
 0.007054153076649019,
 0.001750197149673794,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 7.222999756309822e-06,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 0.037842206111637286,
 6.02910146015634e-05,
 0.36807864522574574,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 5.546664961563126e-05,
 5.844609760329209e-06,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.168084317448003e-18,
 8.441283953122413e-05,
 8.168084317448003e-18,
 2.5142069704057783e-05,
 1.687172972817411e-05,
 4.466219764348598e-06,
 1.1358169744251661e-05,
 5.155414762338904e-06,
 3.3104977640676076e-07,
 2.398634770377679e-06,
 1.020244774397066

In [20]:

index.batch_get_smoothed_probs([tokenizer.encode("hello world"), tokenizer.encode("hello universe")])

[[3.3104977640676076e-07,
  8.168084317448003e-18,
  0.007054153076649019,
  0.001750197149673794,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  7.222999756309822e-06,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  0.037842206111637286,
  6.02910146015634e-05,
  0.36807864522574574,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  5.546664961563126e-05,
  5.844609760329209e-06,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.168084317448003e-18,
  8.441283953122413e-05,
  8.168084317448003e-18,
  2.5142069704057783e-05,
  1.687172972817411e-05,
  4.466219764348598e-06,
  1.1358169744251661e-05,
  5.155414762338904e-06,
  3.3104977640676076e-07,
  